In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

%matplotlib inline

In [ ]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [ ]:
print(pd.__version__)
print(np.__version__)

In [ ]:
%time
df = pd.read_csv('../input/wine-quality/wineQualityWhites.csv' )


## Описание данных

In [ ]:
print('Кол-во строк: ', df.shape[0])
print('Кол-во столбцов: ', df.shape[1])

In [ ]:
#Удалили порядковый номер (1 столбец) так как он не несет смысловой нагрузки
df = df.drop(columns = ['Unnamed: 0'])

Описание переменных:

    - fixed acidity: кислотность вина
    - volatile acidity: количество уксусной кислоты в вине, которая может привести к неприятному вкусу вина
    - citric acid: лимонная кислота, может придать свежесть и вкус вину
    - residual sugar: остаточный сахар (оставшийся после окончания брожения)
    - chlorides: количество солей в вине
    - free sulfur dioxide:свободная форма SO2, предотвращает рост бактерий в вине и его окисление
    - total sulfur dioxide: содержание диоксида серы, может влиять на запах вина
    - density: плотность, зависит от уровня алкоголя и содержания сахара
    - pH: описывает, кислотное или щелочное вино, (шкала от 0 - очень кислотный до 14 - щелочной). В основном, вина имеют рН около 3-4 
    - sulphates: винная добавка, способствующая повышению уровня  диоксида серы (S02),  действует как антимикробный и антиоксидантный элемент
    - alcohol: содержание алкоголя в вине
    - quality : значение качества от 0 дот 10 - целевая переменная

In [ ]:
#Проверим типы данных, а также наличие пропущенных значений
df.info()

Данный датасет не имеет пропущенных значений. Все независимые переменные имеют вещественный тип данных, тогда как целевая переменная принимает только целые значения. 

In [ ]:
#Описательная статистика
df.describe()

In [ ]:
#Oтобразим первые 5 строк
df.head()

## Визуализация данных

In [ ]:
plt.hist(df['fixed.acidity'], bins=np.arange(3, df['fixed.acidity'].max()+1, 1))
plt.xlabel('Fixed acidity')
plt.ylabel('Counts')
plt.title('Histogram of fixed acidity')

In [ ]:
plt.hist(df['total.sulfur.dioxide'], bins=np.arange(0, df['total.sulfur.dioxide'].max()+11, 15))
plt.xlabel('Sulfur dioxide')
plt.ylabel('Counts')
plt.title('Histogram of total sulfur dioxide')

In [ ]:
#Pairplot
sb.pairplot(df)

In [ ]:
#Вывод

На данных графиках отражена попарная зависимость переменных друг от друга, можно наблюдать зависимость переменных residual sugar и density, alcohol и quality, free and total sulfur dioxide, alcohol и density. Проверим выводы согласно pairplot - построим heatmap

In [ ]:
#Heatmap
# вводим новую переменную cor для матрицы корреляций признаков. функция corr считаем корреляцию между признаками.можно описать корреляции,   строим heatmap ,
#fig ax - Обозначили размер картинки
cor = df.corr()
fig, ax = plt.subplots(figsize = (10,10))
sb.heatmap(cor, annot = True, cmap="YlGnBu")

In [ ]:
#Вывод

По диагонали стоят 1 - поскольку показана корелляция переменной с самой собой. Наибольшую корреляцию с целевой переменной имеет переменная alcohol

In [ ]:
#Построим ящик с усами (boxplot)
sb.boxplot(x = df['quality'], y = df['alcohol'])

In [ ]:
#Вывод: для вин более высокого качества характерно повышенное содержание алкоголя

Эта зависимость прослеживается в диапазоне качеста от 5 до 9.
Точки - это выбросы, отрезки - границы квартилей

In [ ]:
# отбираем количественные признаки (выбрали только : - выбрали все строки, запятая, потом какие столбцы нужны - выбираем минус 1, iloc - функция помогает показать какой фрагмент данных нужно взять)
x = df.iloc[:,1:12]

##здесь мы положили только x в train model (наши регрессоры)

In [ ]:
y = df[['quality']]

In [ ]:
x.head()

In [ ]:
y

In [ ]:
#Нормализация данных
scaler = MinMaxScaler(feature_range=(0, 1))
x = scaler.fit_transform(x)

Посмотрев на описательную статистику, мы можем заметить, что разные переменные имеют разную размерность
Проведем нормализацию данных - все признаки теперь имеют значение от 0 до 1

In [ ]:
#Разделим выборку на обучающую и тестовую в отношении 80 и 20 %
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

## Логистическая регрессия

In [ ]:
#Аргументы функции:
# - multi_class = 'multinomial' - уточняем, что в отличии от стандартной бинарной логистической регрессии в данном случае целевая переменная может принимать более 2 значений
# - cv = 3- столько раз мы разделим данные на обучающую и тестовую выборку, обучим и протестируем модель

best_clf_LR = LogisticRegressionCV(multi_class = 'multinomial', cv = 3)

#Обучаем модель
best_clf_LR.fit(x_train, y_train)

In [ ]:
#Получаем прогнозируемые значения
y_pred_LR = best_clf_LR.predict(x_test)

In [ ]:
#Получим точность модели
print(accuracy_score(y_pred_LR, y_test))

In [ ]:
#Confusion matrix
cm_LR = confusion_matrix(y_pred_LR, y_test)
cm_LR

In [ ]:
#Вывод

# RandomForest

In [ ]:
# Создаем случайный набор гиперпараметров для модели
param_grid = {
    'max_depth': [80, 90, 100, 110],
    'max_features': [10, 11],
    'min_samples_leaf': [10, 20, 50],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

Параметры - случайные,  max depth - глубина, на которую может делиться дерево
max features - скольк переменных включены в модель 
n estimators - количество деревьей

In [ ]:
#Обучаем модель
best_clf_RF = RandomForestClassifier()
grid_search_RF = GridSearchCV(best_clf_RF, param_grid = param_grid, cv = 3)
grid_search_RF.fit(x_train, y_train)

In [ ]:
#Получим точность модели
np.mean(grid_search_RF.cv_results_['mean_test_score'])

In [ ]:
#Вывод

# XGboost

In [ ]:
# Создаем случайный набор гиперпараметров для модели
param_grid = {
    'max_depth': [80, 90, 100, 110],
    'n_estimators': [100, 200, 300, 1000]
}

In [ ]:
#Обучаем модель
best_clf_XG = XGBClassifier()
grid_search_XG = GridSearchCV(best_clf_XG, param_grid = param_grid, cv = 3)
grid_search_XG.fit(x_train, y_train)

In [ ]:
#Получим точность модели
np.mean(grid_search_XG.cv_results_['mean_test_score'])

In [ ]:
#Вывод